## Multipole covariance

In [1]:
import sys
sys.path.insert(1, '/home/oalves/thecov')
import thecov.base
import numpy as np

In [15]:
matrix_a = np.array([[1, 2], [3, 4]])
matrix_b = np.array([[5, 6, 7], [8, 9, 10], [11,12,13]])
matrix_c = np.array([[9, 10], [11, 12], [13,14]])
matrix_d = np.array([[13, 14, 15], [16, 17, 18]])


In [16]:
cov = thecov.base.MultipoleCovariance(symmetric=False)

In [19]:
cov.set_ell_cov(0,0,thecov.base.Covariance(matrix_a))
cov.set_ell_cov(2,2,thecov.base.Covariance(matrix_b))
cov.set_ell_cov(0,2,thecov.base.Covariance(matrix_d))
cov.set_ell_cov(2,0,thecov.base.Covariance(matrix_c))

In [20]:
cov.cov

array([[ 1,  2, 13, 14, 15],
       [ 3,  4, 16, 17, 18],
       [ 9, 10,  5,  6,  7],
       [11, 12,  8,  9, 10],
       [13, 14, 11, 12, 13]])

In [21]:
cov.cov = cov.cov

AssertionError: Covariance matrix shape should be a multiple of the number of ells.

In [12]:
cov.cov

array([[ 1,  2, 13, 15],
       [ 3,  4, 14, 16],
       [13, 14,  5,  6],
       [15, 16,  7,  8]])

In [13]:
cov._multipole_covariance

{(0, 0): <thecov.base.Covariance at 0x735d8c167550>,
 (2, 2): <thecov.base.Covariance at 0x735d8c1668c0>,
 (0, 2): <thecov.base.Covariance at 0x735d8c1653c0>}

In [19]:
cov.ells = (0,2), (0,2,4)

In [20]:
cov._multipole_covariance

{(0,
  0): array([[0, 0],
        [0, 0]]),
 (0, 2): None,
 (0, 4): None,
 (2, 2): None,
 (2, 4): None}

## Gaunt coefficients

In [1]:
import scipy
import sympy.physics.wigner
import numpy as np
import itertools as itt
import sys
sys.path.insert(1, '/home/oalves/thecov')
import thecov.base

In [2]:
# Define the shape of the multi-dimensional sparse array
shape_out = (3, 3, 3, 3, 5, 5, 5, 5) # l1, l2, l3, l4, m1, m2, m3, m4
shape_in = (7, 7, 13, 13) # la, lb, ma, mb

In [ ]:
coefficients = thecov.base.SparseNDArray(shape_out, shape_in)

for l1, l2, l3, l4 in itt.product((0,2,4), repeat=4):
    for m1, m2, m3, m4 in itt.product(*[np.arange(-l, l+1, 2) for l in (l1, l2, l3, l4)]):
        for la in np.arange(np.abs(l1-l4), l1+l4+1, 2):
            for lb in np.arange(np.abs(l2-l3), l2+l3+1, 2):
                for ma, mb in itt.product(*[np.arange(-l, l+1, 2) for l in (la, lb)]):
                    value = np.float64(sympy.physics.wigner.gaunt(l1,l4,la,m1,m4,ma)*\
                                       sympy.physics.wigner.gaunt(l2,l3,lb,m2,m3,mb))
                    if value != 0.:
                        coefficients[l1//2,l2//2,l3//2,l4//2,
                                     m1//2+2,m2//2+2,m3//2+2,m4//2+2,
                                     la//2,lb//2,
                                     ma//2+6,mb//2+6] += value
                        
        for lc in np.arange(np.abs(l1-l2), l1+l2+1, 2):
            for la in np.arange(np.abs(lc-l4), lc+l4+1, 2):
                for ma, mc in itt.product(*[np.arange(-l, l+1, 2) for l in (la, lc)]):
                    value = np.float64(sympy.physics.wigner.gaunt(l1,l2,lc,m1,m2,mc)*\
                                       sympy.physics.wigner.gaunt(lc,l4,la,mc,m4,ma))
                    lb, mb = l3, m3
                    if value != 0.:
                        coefficients[l1//2,l2//2,l3//2,l4//2,
                                     m1//2+2,m2//2+2,m3//2+2,m4//2+2,
                                     la//2,lb//2,
                                     ma//2+6,mb//2+6] += value
coefficients

/home/oalves/software/anaconda3/lib/python3.10/site-packages/scipy/sparse/_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


SparseNDArray(shape_out=[3 3 3 3 5 5 5 5] -> 50625, shape_in=[ 7  7 13 13] -> 8281, nnz=60125)

In [4]:
coefficients.save('cosmic_variance_coefficients.npz')

In [3]:
coefficients = thecov.base.SparseNDArray.load('cosmic_variance_coefficients.npz')
coefficients

SparseNDArray(shape_out=[3 3 3 3 5 5 5 5] -> 50625, shape_in=[ 7  7 13 13] -> 8281, nnz=60125)

In [5]:
(coefficients @ np.ones((*shape_in, 3))).shape

(50625, 3)